In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '90_reporting_and_codex_mcp'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Lab 90 · Reporting, Codex, and MCP

We generate a deterministic HTML report, document optional PDF conversion, and
explore Codex/MCP tooling with dry-run friendly guards.


## Synthetic Case Summary
A compact dataset feeds the report exporter. It references prior labs so the
report feels realistic without depending on live cases.


In [ ]:
REPORT_DIR = LAB_ROOT / 'reports'
REPORT_DIR.mkdir(parents=True, exist_ok=True)

REPORT_TS = _ts()
report_data = {
    'case': {
        'case_id': 'CASE-LAB-90',
        'name': 'Notebook Investigation Demo',
        'investigator': 'Notebook Analyst',
        'summary': 'Synthetic case data demonstrating report generation.',
    },
    'executive_summary': {
        'timeline': 'Host contacted evil-c2.onion and transferred funds.',
        'impact': 'Outbound beacon detected and contained within 15 minutes.',
    },
    'findings': [
        {
            'title': 'Router Pipeline Summary',
            'details': 'Router export analysed with manifest and Markdown summary.',
            'severity': 'medium',
        },
        {
            'title': 'IoC Matches',
            'details': 'Indicators matched in firewall.log and beacon.txt.',
            'severity': 'high',
        },
    ],
    'timeline': {
        'events': [
            {'ts': '2024-03-01T12:03:12Z', 'event': 'Firewall blocked outbound HTTPS to 198.51.100.23'},
            {'ts': '2024-03-01T12:04:45Z', 'event': 'Dynamic DNS updated to blue-gateway.example.net'},
            {'ts': '2024-03-01T12:05:44Z', 'event': 'SSH access from analyst workstation'},
        ]
    },
    'evidence': [
        {'path': 'analysis/router/manifest.json', 'description': 'Router manifest with SHA256 hashes'},
        {'path': 'analysis/ioc_scan/ioc_scan_results.json', 'description': 'Deterministic IoC scan output'},
    ],
    'chain_of_custody': [
        {'ts': '2024-03-01T12:02:00Z', 'action': 'Evidence intake', 'actor': 'Notebook Analyst'},
        {'ts': '2024-03-01T12:06:00Z', 'action': 'Router archive hashed', 'actor': 'Notebook Analyst'},
    ],
}

json_dump_sorted(report_data, LAB_ROOT / f'{REPORT_TS}_report_data.json')
report_data


## Generate HTML Report (SDK-first)
`export_report` from the reporting module handles deterministic rendering.
The exporter falls back to a built-in template when Jinja2 is unavailable.


In [ ]:
from forensic.modules.reporting.exporter import export_report, get_pdf_renderer, export_pdf

html_path = export_report(report_data, 'html', REPORT_DIR / f'{REPORT_TS}_report.html')
markdown_path = export_report(report_data, 'md', REPORT_DIR / f'{REPORT_TS}_report.md')
{'html': str(html_path), 'markdown': str(markdown_path)}


### Optional PDF Guard
If a PDF renderer (wkhtmltopdf or WeasyPrint) is installed we convert the HTML
report. Otherwise we record a guard message.


In [ ]:
renderer = get_pdf_renderer()
pdf_status = {'renderer': renderer, 'generated': False}
if renderer:
    pdf_path = export_pdf(html_path, REPORT_DIR / f'{REPORT_TS}_report.pdf')
    pdf_status['generated'] = True
    pdf_status['pdf_path'] = str(pdf_path)
else:
    pdf_status['hint'] = 'Install wkhtmltopdf or weasyprint to enable PDF export'
json_dump_sorted(pdf_status, LAB_ROOT / f'{REPORT_TS}_pdf_status.json')
pdf_status


### CLI Mirror (optional)
For documentation we capture the CLI command that would generate the same
report. We keep the invocation in dry-run mode to avoid touching real cases.


In [ ]:

if CLI_OK:
    cli_command = [
        'forensic-cli', 'report', 'generate',
        '--case', 'CASE-LAB-90',
        '--fmt', 'html',
        '--dry-run',
    ]
    cli_result = run_cli(cli_command)
    cli_record = {
        'command': cli_command,
        'executed': True,
        'returncode': cli_result.returncode,
        'stdout': cli_result.stdout.strip(),
        'stderr': cli_result.stderr.strip(),
    }
else:
    cli_record = {
        'command': [
            'forensic-cli', 'report', 'generate', '--case', 'CASE-LAB-90', '--fmt', 'html', '--dry-run'
        ],
        'executed': False,
        'hint': 'forensic-cli not available; record the command for later confirmation.',
    }
json_dump_sorted(cli_record, LAB_ROOT / 'report_cli_plan.json')
cli_record


## Codex / MCP Dry-Run Plan
Codex orchestrates MCP tools. We keep actions in preview mode to honour forensic
mode guardrails.


In [ ]:

codex_plan = {
    'steps': [
        {
            'label': 'Install Codex packages',
            'command': ['forensic-cli', 'codex', 'install', '--dry-run'],
            'mode': 'dry-run-only',
        },
        {
            'label': 'Start Codex in foreground',
            'command': ['forensic-cli', 'codex', 'start', '--foreground'],
            'mode': 'plan-only',
            'confirm_gate': 'Require analyst confirmation before removing --dry-run guard.',
        },
        {
            'label': 'Expose MCP catalogue',
            'command': ['forensic-cli', 'mcp', 'expose', '--json'],
            'mode': 'collect-artifacts',
        },
    ],
}
json_dump_sorted(codex_plan, LAB_ROOT / 'codex_plan.json')
codex_plan


### MCP Catalogue
We attempt to expose the MCP tool catalogue via CLI when available. Otherwise
we persist a deterministic sample catalogue.


In [ ]:
MCP_CATALOG_PATH = LAB_ROOT / 'mcp_catalog.json'
if CLI_OK:
    result = run_cli(['forensic-cli', 'mcp', 'expose', '--json'])
    if result.returncode == 0 and result.stdout:
        try:
            catalogue = json.loads(result.stdout)
        except json.JSONDecodeError:
            catalogue = {'status': 'error', 'message': 'Failed to decode CLI output'}
    else:
        catalogue = {
            'status': 'guarded',
            'message': 'CLI invocation unsuccessful; falling back to synthetic catalogue',
            'stderr': result.stderr.strip(),
        }
else:
    catalogue = {
        'status': 'synthetic',
        'tools': [
            {'name': 'diagnostics.ping', 'description': 'Local connectivity check'},
            {'name': 'modules.list', 'description': 'Enumerate registered modules'},
        ],
    }
json_dump_sorted(catalogue, MCP_CATALOG_PATH)
catalogue


## Forensic Mode Prompt & Confirm-Gate
Codex sessions run with the forensic-mode system prompt. We load the prompt
snippet and define a confirm-gate policy example for potentially destructive
commands.


In [ ]:
prompt_path = Path('forensic/mcp/prompts/forensic_mode.txt')
forensic_prompt = prompt_path.read_text(encoding='utf-8').splitlines()[:6]

confirm_gate = {
    'trigger': 'Commands touching live evidence or network state',
    'policy': 'Require analyst confirmation with case ID and intended impact before execution.',
    'example_plan': [
        'Plan: Run diagnostics.ping against 198.51.100.23 to confirm reachability.',
        'Gate: Analyst confirms scope and records output path in case meta.',
        'Execute: Perform diagnostics.ping --target 198.51.100.23 --output meta/ping.log',
    ],
}
preview_payload = {
    'prompt_preview': forensic_prompt,
    'confirm_gate': confirm_gate,
}
json_dump_sorted(preview_payload, LAB_ROOT / 'forensic_mode_preview.json')
preview_payload


### Checkpoint
Reports and MCP artefacts should be present even when CLI tooling is absent.


In [ ]:

expected = [
    html_path,
    markdown_path,
    MCP_CATALOG_PATH,
    LAB_ROOT / 'codex_plan.json',
    LAB_ROOT / 'forensic_mode_preview.json',
    LAB_ROOT / 'report_cli_plan.json',
]
for item in expected:
    assert Path(item).exists(), f'Missing artefact: {item}'

LAB_STATUS = {
    'report_html': str(html_path),
    'mcp_catalog': str(MCP_CATALOG_PATH),
    'pdf_generated': pdf_status.get('generated', False),
}
json_dump_sorted(LAB_STATUS, LAB_ROOT / 'lab_status.json')
LAB_STATUS
